In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from pandas_datareader import data as pdr
import yfinance as yf
from keras.models import load_model
import datetime
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

yf.pdr_override()

today = datetime.date.today()

start = today - relativedelta(years=5)
end = today

user_input = 'AAPL'


df = pdr.get_data_yahoo(user_input,start,end)
df = df.reset_index()
date = df['Date']

df = df[['Date','Close']]
df

[*********************100%%**********************]  1 of 1 completed


,Date,Close
0,2019-01-10,38.450001
1,2019-01-11,38.072498
2,2019-01-14,37.500000
3,2019-01-15,38.267502
4,2019-01-16,38.735001
...,...,...
1253,2024-01-03,184.250000
1254,2024-01-04,181.910004
1255,2024-01-05,181.179993
1256,2024-01-08,185.559998


In [2]:
df['day_of_week'] = df['Date'].dt.dayofweek 
#df["Start_of_year"], quarter, other featurues 

In [3]:
data_training = pd.DataFrame(df['Close'][0:int(len(df)*0.7)])
data_testing = pd.DataFrame(df['Close'][int(len(df)*0.7): int(len(df))])

In [4]:
scalar =  MinMaxScaler(feature_range=(0,1))
data_training_array = scalar.fit_transform(data_training)
#load keras model 
model = load_model('keras_model.h5')

In [5]:
# testing part 
past_100_days= data_training.tail(100)
# the 'feautures' to use to predict the values 



final_df = pd.concat([past_100_days, data_testing],ignore_index = True)
#null_df = [None, None, None, None, None]
#final_df = pd.concat([final_df,np.array(null_df)], ignore_index=True)
input_data = scalar.fit_transform(final_df)



X_test = []
y_test = []
# start from day 100 and go till the end of the data
for i in range(100,input_data.shape[0]): 
    X_test.append(input_data[i-100:i])
    # X_test will consist of the array that will be the past 100 days for i's date 
    y_test.append(input_data[i,0])
    # y_test will have the value of the close price of ith day 


X_test,y_test = np.array(X_test),np.array(y_test)
print(y_test)

[2.71582944e-01 2.85127963e-01 2.80065778e-01 3.20837369e-01
 3.44096322e-01 3.01682987e-01 3.55452209e-01 3.83362910e-01
 4.14967955e-01 3.97728802e-01 3.82131602e-01 3.63661349e-01
 4.34669516e-01 4.42331478e-01 5.12929208e-01 4.99247447e-01
 4.78724805e-01 5.62457327e-01 5.58079063e-01 5.51785569e-01
 5.45218174e-01 5.45902303e-01 6.05007613e-01 5.94746292e-01
 6.44137459e-01 6.59050529e-01 6.56861397e-01 6.77657733e-01
 6.72184903e-01 6.36202013e-01 5.82159097e-01 5.77507139e-01
 5.81611709e-01 6.15816113e-01 5.28115972e-01 4.97468960e-01
 4.63674992e-01 4.40552782e-01 4.50677361e-01 4.21261448e-01
 4.03748810e-01 4.23313838e-01 4.02791195e-01 4.42604963e-01
 5.25516404e-01 3.94308361e-01 4.14420567e-01 3.74196155e-01
 3.51347639e-01 4.03064680e-01 4.36174518e-01 3.92666617e-01
 3.79258549e-01 3.47653505e-01 3.52305462e-01 3.65850272e-01
 3.39581315e-01 2.38883543e-01 1.80325620e-01 2.38473107e-01
 2.88411661e-01 2.92516023e-01 2.79244697e-01 2.06184141e-01
 2.10699147e-01 1.909973

In [6]:
y_predicted = model.predict(X_test)
scale_factor = 1/scalar.scale_
y_predicted = y_predicted * scale_factor
y_test = y_test * scale_factor

x =user_input +  ' closing price predictions vs original in the past'

12/12 [==============================] - 1s 32ms/step


In [7]:
y_test

array([1.98499985e+01, 2.08400040e+01, 2.04700089e+01, 2.34500046e+01,
       2.51500015e+01, 2.20500107e+01, 2.59800034e+01, 2.80199966e+01,
       3.03300095e+01, 2.90699997e+01, 2.79300003e+01, 2.65800095e+01,
       3.17699966e+01, 3.23300095e+01, 3.74899979e+01, 3.64899979e+01,
       3.49899979e+01, 4.11100082e+01, 4.07900009e+01, 4.03300095e+01,
       3.98499985e+01, 3.99000015e+01, 4.42200089e+01, 4.34700089e+01,
       4.70800095e+01, 4.81700058e+01, 4.80100021e+01, 4.95300064e+01,
       4.91299973e+01, 4.65000076e+01, 4.25500107e+01, 4.22099991e+01,
       4.25100021e+01, 4.50100021e+01, 3.85999985e+01, 3.63600082e+01,
       3.38900070e+01, 3.22000046e+01, 3.29400101e+01, 3.07900009e+01,
       2.95100021e+01, 3.09400101e+01, 2.94400101e+01, 3.23499985e+01,
       3.84099960e+01, 2.88199997e+01, 3.02900009e+01, 2.73499985e+01,
       2.56800003e+01, 2.94599991e+01, 3.18799973e+01, 2.87000046e+01,
       2.77200089e+01, 2.54099960e+01, 2.57500076e+01, 2.67399979e+01,
      

In [11]:
df 

,Date,Close,day_of_week
0,2019-01-10,38.450001,3
1,2019-01-11,38.072498,4
2,2019-01-14,37.500000,0
3,2019-01-15,38.267502,1
4,2019-01-16,38.735001,2
...,...,...,...
1253,2024-01-03,184.250000,2
1254,2024-01-04,181.910004,3
1255,2024-01-05,181.179993,4
1256,2024-01-08,185.559998,0


In [8]:
cvscores = []
scores = model.evaluate(X_test, y_test/scale_factor, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
cvscores.append(scores[1] * 100)
print("%.2f%% (+- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

accuracy: 0.53%
0.53% (+- 0.00%)


In [9]:
y_predicted

array([[13.44721569],
       [13.8153751 ],
       [14.25571743],
       [14.78464926],
       [15.46939471],
       [16.32905488],
       [17.30383783],
       [18.33549386],
       [19.41168969],
       [20.54963242],
       [21.73909952],
       [22.92615094],
       [24.03350448],
       [25.06886898],
       [26.06036131],
       [27.08985977],
       [28.19510912],
       [29.34618454],
       [30.57096764],
       [31.86930159],
       [33.20168597],
       [34.50399306],
       [35.71521436],
       [36.85962433],
       [37.94789203],
       [39.03746232],
       [40.17897521],
       [41.3789698 ],
       [42.63717164],
       [43.92115091],
       [45.13064269],
       [46.09444072],
       [46.70523125],
       [46.95196184],
       [46.92580104],
       [46.61809247],
       [46.01313529],
       [45.12229563],
       [43.98487349],
       [42.66829015],
       [41.23046966],
       [39.71880208],
       [38.19583372],
       [36.70864098],
       [35.31672061],
       [34

In [10]:
past_100_days = (y_test / scale_factor)[-100:]
past_100_days = past_100_days.reshape(1,100,1)

In [11]:
y_test.shape

(378,)

In [12]:
y_test = y_test/scale_factor

In [13]:
# day 1 
for i in range(1,6):
    past_100_days = y_test[-100:]

    
    past_100_days = past_100_days.reshape(1,100,1)  
    day = model.predict(past_100_days)
    print(day.shape)
    day = day.reshape(1,1)
    y_test = np.append(y_test, day)

1/1 [==============================] - 0s 19ms/step
(1, 1)
1/1 [==============================] - 0s 21ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)
1/1 [==============================] - 0s 20ms/step
(1, 1)


In [14]:
print((y_test*scale_factor)[-5:])

[61.06450713 59.64556337 58.45988861 57.54232179 56.88417171]


In [16]:
y_test * scale_factor

array([1.98499985e+01, 2.08400040e+01, 2.04700089e+01, 2.34500046e+01,
       2.51500015e+01, 2.20500107e+01, 2.59800034e+01, 2.80199966e+01,
       3.03300095e+01, 2.90699997e+01, 2.79300003e+01, 2.65800095e+01,
       3.17699966e+01, 3.23300095e+01, 3.74899979e+01, 3.64899979e+01,
       3.49899979e+01, 4.11100082e+01, 4.07900009e+01, 4.03300095e+01,
       3.98499985e+01, 3.99000015e+01, 4.42200089e+01, 4.34700089e+01,
       4.70800095e+01, 4.81700058e+01, 4.80100021e+01, 4.95300064e+01,
       4.91299973e+01, 4.65000076e+01, 4.25500107e+01, 4.22099991e+01,
       4.25100021e+01, 4.50100021e+01, 3.85999985e+01, 3.63600082e+01,
       3.38900070e+01, 3.22000046e+01, 3.29400101e+01, 3.07900009e+01,
       2.95100021e+01, 3.09400101e+01, 2.94400101e+01, 3.23499985e+01,
       3.84099960e+01, 2.88199997e+01, 3.02900009e+01, 2.73499985e+01,
       2.56800003e+01, 2.94599991e+01, 3.18799973e+01, 2.87000046e+01,
       2.77200089e+01, 2.54099960e+01, 2.57500076e+01, 2.67399979e+01,
      